# Pytorch Lighting with Ray

The RayPlugin provides `Distributed Data Parallel` training on a Ray cluster. PyTorch DDP is used as the distributed training protocol, and Ray is used to launch and manage the training worker processes.


in this notebook we will reproduce the training in 
https://colab.research.google.com/github/PyTorchLightning/lightning-tutorials/blob/publication/.notebooks/lightning_examples/mnist-hello-world.ipynb
 

also you can find more detail about DDP in https://pytorch.org/tutorials/intermediate/ddp_tutorial.html

source code of ddp implementation in ray: https://docs.ray.io/en/master/_modules/ray_lightning/ray_ddp.html



In [2]:
import pytorch_lightning as pl
from ray_lightning import RayPlugin

In [3]:
import os

import torch
from pytorch_lightning import LightningModule, Trainer
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
from torchmetrics import Accuracy
from torchvision import transforms
from torchvision.datasets import MNIST

PATH_DATASETS = os.environ.get("PATH_DATASETS", ".")
BATCH_SIZE = 32

In [4]:

class LitMNIST(LightningModule):
    def __init__(self, data_dir=PATH_DATASETS, hidden_size=64, learning_rate=2e-4):

        super().__init__()

        # Set our init args as class attributes
        self.data_dir = data_dir
        self.hidden_size = hidden_size
        self.learning_rate = learning_rate

        # Hardcode some dataset specific attributes
        self.num_classes = 10
        self.dims = (1, 28, 28)
        channels, width, height = self.dims
        self.transform = transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.Normalize((0.1307,), (0.3081,)),
            ]
        )

        # Define PyTorch model
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(channels * width * height, hidden_size),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(hidden_size, self.num_classes),
        )

        self.accuracy = Accuracy()

    def forward(self, x):
        x = self.model(x)
        return F.log_softmax(x, dim=1)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        preds = torch.argmax(logits, dim=1)
        self.accuracy(preds, y)

        # Calling self.log will surface up scalars for you in TensorBoard
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", self.accuracy, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        # Here we just reuse the validation_step for testing
        return self.validation_step(batch, batch_idx)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

    ####################
    # DATA RELATED HOOKS
    ####################

    def prepare_data(self):
        # download
        MNIST(self.data_dir, train=True, download=True)
        MNIST(self.data_dir, train=False, download=True)

    def setup(self, stage=None):

        # Assign train/val datasets for use in dataloaders
        if stage == "fit" or stage is None:
            mnist_full = MNIST(self.data_dir, train=True, transform=self.transform)
            self.mnist_train, self.mnist_val = random_split(mnist_full, [55000, 5000])

        # Assign test dataset for use in dataloader(s)
        if stage == "test" or stage is None:
            self.mnist_test = MNIST(self.data_dir, train=False, transform=self.transform)

    def train_dataloader(self):
        return DataLoader(self.mnist_train, batch_size=BATCH_SIZE)

    def val_dataloader(self):
        return DataLoader(self.mnist_val, batch_size=BATCH_SIZE)

    def test_dataloader(self):
        return DataLoader(self.mnist_test, batch_size=BATCH_SIZE)

In [5]:
plugin = RayPlugin(num_workers=8, num_cpus_per_worker=1)

# Don't set ``gpus`` in the ``Trainer``.
# The actual number of GPUs is determined by ``num_workers``.

model = LitMNIST()
trainer = Trainer(
    gpus=0,
    max_epochs=3,
    progress_bar_refresh_rate=20,
    plugins=[plugin]  # <- only add this line
)
trainer.fit(model)

2021-12-16 18:21:34,557	INFO services.py:1340 -- View the Ray dashboard at http://127.0.0.1:8265
2021-12-16 18:21:34,570	WARNING services.py:1826 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=5.04gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
(RayExecutor pid=176) initializing ddp: GLOBAL_RANK: 0, MEMBER: 1/8
(RayExecutor pid=180) initializing ddp: GLOBAL_RANK: 4, MEMBER: 5/8
(RayExecutor pid=178) initializing ddp: GLOBAL_RANK: 7, MEMBER: 8/8
(RayExecutor pid=179) initializing ddp: GLOBAL_RANK: 3, MEMBER: 4/8
(RayExecutor pid=177) initiali

Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]


(RayExecutor pid=176) /usr/local/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:106: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
(RayExecutor pid=176)   f"The dataloader, {name}, does not have many workers which may be a bottleneck."
(RayExecutor pid=181) [W reducer.cpp:1303] Warning: find_unused_parameters=True was specified in DDP constructor, but did not find any unused parameters in the forward pass. This flag results in an extra traversal of the autograd graph every iteration,  which can adversely affect performance. If your model indeed never has any unused parameters in the forward pass, consider turning this flag off. Note that this warning may be a false positive if your model has flow control causing later iterations to have unused parame

                                                              
Training: -1it [00:00, ?it/s]) 
Epoch 0:   0%|          | 0/235 [00:00<00:00, 2460.00it/s]  


(RayExecutor pid=180) [W reducer.cpp:1303] Warning: find_unused_parameters=True was specified in DDP constructor, but did not find any unused parameters in the forward pass. This flag results in an extra traversal of the autograd graph every iteration,  which can adversely affect performance. If your model indeed never has any unused parameters in the forward pass, consider turning this flag off. Note that this warning may be a false positive if your model has flow control causing later iterations to have unused parameters. (function operator())
(RayExecutor pid=182) [W reducer.cpp:1303] Warning: find_unused_parameters=True was specified in DDP constructor, but did not find any unused parameters in the forward pass. This flag results in an extra traversal of the autograd graph every iteration,  which can adversely affect performance. If your model indeed never has any unused parameters in the forward pass, consider turning this flag off. Note that this warning may be a false positive i

Epoch 0:  94%|█████████▎| 220/235 [00:13<00:00, 16.83it/s, loss=0.559, v_num=6]
Validating: 0it [00:00, ?it/s]m 
Validating:   0%|          | 0/20 [00:00<?, ?it/s]
(RayExecutor pid=176) 
Epoch 1:  94%|█████████▎| 220/235 [00:11<00:00, 19.62it/s, loss=0.366, v_num=6, val_loss=0.447, val_acc=0.886]
Validating: 0it [00:00, ?it/s]m 
Validating:   0%|          | 0/20 [00:00<?, ?it/s]
(RayExecutor pid=176) 
Epoch 1: 100%|██████████| 235/235 [00:12<00:00, 19.05it/s, loss=0.328, v_num=6, val_loss=0.311, val_acc=0.914]
                                                           
Epoch 2:  94%|█████████▎| 220/235 [00:11<00:00, 18.92it/s, loss=0.311, v_num=6, val_loss=0.311, val_acc=0.914]
Validating: 0it [00:00, ?it/s]m 
Validating:   0%|          | 0/20 [00:00<?, ?it/s]
(RayExecutor pid=176) 
Epoch 2: 100%|██████████| 235/235 [00:12<00:00, 19.09it/s, loss=0.357, v_num=6, val_loss=0.263, val_acc=0.923]


In [13]:
trainer.test(model)

(RayExecutor pid=3140) initializing ddp: GLOBAL_RANK: 0, MEMBER: 1/8
(RayExecutor pid=3146) initializing ddp: GLOBAL_RANK: 6, MEMBER: 7/8
(RayExecutor pid=3145) initializing ddp: GLOBAL_RANK: 5, MEMBER: 6/8
(RayExecutor pid=3141) initializing ddp: GLOBAL_RANK: 1, MEMBER: 2/8
(RayExecutor pid=3147) initializing ddp: GLOBAL_RANK: 7, MEMBER: 8/8
(RayExecutor pid=3143) initializing ddp: GLOBAL_RANK: 3, MEMBER: 4/8
(RayExecutor pid=3142) initializing ddp: GLOBAL_RANK: 2, MEMBER: 3/8
(RayExecutor pid=3144) initializing ddp: GLOBAL_RANK: 4, MEMBER: 5/8
(RayExecutor pid=3140) /usr/local/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:106: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
(RayExecutor pid=3140)   f"The dataloader, {name}, does not have many worke

Testing: 0it [00:00, ?it/s]140) 
Testing:  50%|█████     | 20/40 [00:00<00:00, 39.71it/s]


[{'val_loss': 0.2514459788799286, 'val_acc': 0.9279999732971191}]

Testing: 100%|██████████| 40/40 [00:01<00:00, 35.69it/s]--------------------------------------------------------------------------------
(RayExecutor pid=3140) DATALOADER:0 TEST RESULTS
(RayExecutor pid=3140) {'val_acc': 0.9279999732971191, 'val_loss': 0.2514459788799286}
(RayExecutor pid=3140) --------------------------------------------------------------------------------
Testing: 100%|██████████| 40/40 [00:01<00:00, 35.42it/s]
